# UDA-Hub: Multi-Agent Customer Support System

This notebook demonstrates the LangGraph-based multi-agent system for CultPass customer support.

**Architecture**: Hierarchical (Supervisor) pattern with 4 agents

**Prerequisites:**
- Run `01_external_db_setup.py` first
- Run `02_core_db_setup.ipynb` first
- Have `.env` file with `OPENAI_API_KEY`

## 1. Setup & Imports

In [ ]:
# Load environment variables FIRST
from dotenv import load_dotenv
load_dotenv()

import logging
logging.getLogger().setLevel(logging.INFO)

print("✅ Environment loaded")

In [ ]:
# Import the orchestrator and utilities
from agentic.workflow import orchestrator
from langchain_core.messages import HumanMessage, AIMessage
from utils import chat_interface

print("✅ Orchestrator loaded")
print("\nWorkflow nodes:", list(orchestrator.get_graph().nodes.keys()))
print("\nAgents: Classifier, Account Lookup, Resolver, Escalation")

## 2. Helper Function

In [ ]:
def ask(message: str, thread_id: str = "notebook-1", ticket_id: str = "") -> dict:
    """
    Send a message to the support agent and get a response.
    
    Args:
        message: User's question or issue
        thread_id: Session ID for short-term memory
        ticket_id: Ticket ID for long-term memory persistence
    """
    result = orchestrator.invoke(
        input={
            "messages": [HumanMessage(content=message)],
            "ticket_id": ticket_id or thread_id,
        },
        config={"configurable": {"thread_id": thread_id}}
    )
    
    return {
        "response": result["messages"][-1].content,
        "category": result.get("category", "unknown"),
        "escalated": result.get("needs_escalation", False),
        "confidence": result.get("knowledge_confidence", 1.0)
    }

print("✅ Helper function ready")

## 3. Test: All 4 Agents

In [ ]:
# Test Classifier Agent
from agentic.agents.classifier import classify_ticket
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("=" * 50)
print("AGENT 1: CLASSIFIER")
print("=" * 50)

test_msgs = ["I forgot my password", "My account is blocked", "I want a refund"]
for msg in test_msgs:
    result = classify_ticket(msg, model)
    print(f"'{msg}' → {result['category']} (escalate: {result['needs_escalation']})")

In [ ]:
# Test Account Lookup Agent
from agentic.agents.account_lookup import lookup_user_context, get_user_by_email

print("=" * 50)
print("AGENT 2: ACCOUNT LOOKUP")
print("=" * 50)

# Try to look up a user (will work if 01_external_db_setup was run)
context = lookup_user_context("alice@example.com")
print(f"User context for alice@example.com:\n{context}")

In [ ]:
# Test Resolver Agent
from agentic.agents.resolver import generate_response

print("=" * 50)
print("AGENT 3: RESOLVER")
print("=" * 50)

response = generate_response(
    user_message="How do I reset my password?",
    category="login_access",
    knowledge_context="To reset password: Go to login, click Forgot Password, enter email.",
    user_context="",
    model=model
)
print(response)

In [ ]:
# Test Escalation Agent
from agentic.agents.escalation import generate_escalation_response

print("=" * 50)
print("AGENT 4: ESCALATION")
print("=" * 50)

response = generate_escalation_response(
    user_message="My account is blocked!",
    reason="account_blocked",
    user_context="",
    model=model
)
print(response)

## 4. Test: Resolver Path (with Logging)

In [ ]:
# Test login issue - watch the logs!
print("Testing: Login issue")
print("-" * 40)
result = ask("I can't log into my account")

print(f"\nCategory: {result['category']}")
print(f"Confidence: {result['confidence']:.0%}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response'][:500]}...")

## 5. Test: Confidence-Based Escalation (Criterion 5)

When RAG returns low confidence matches, the system automatically escalates.

In [ ]:
# This obscure query should have LOW confidence and trigger escalation
print("Testing: Obscure query (should have low KB match)")
print("-" * 40)

result = ask("I need help with the partnership API integration for corporate accounts")

print(f"\nCategory: {result['category']}")
print(f"Confidence: {result['confidence']:.0%}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
# Compare with a common query (high confidence)
print("Testing: Common query (should have high KB match)")
print("-" * 40)

result = ask("How do I cancel my reservation?")

print(f"\nCategory: {result['category']}")
print(f"Confidence: {result['confidence']:.0%}")
print(f"Escalated: {result['escalated']}")

## 6. Test: Long-Term Memory (Criterion 7 & 8)

Demonstrates cross-session memory persistence using the TicketMessage table.

In [ ]:
# First, let's create a ticket and have a conversation
import uuid

# Create a unique ticket ID for this test
test_ticket_id = str(uuid.uuid4())
print(f"Test Ticket ID: {test_ticket_id}")
print("=" * 50)

# Message 1
print("\n[Session 1] User: I'm having issues with my premium subscription")
result1 = ask(
    "I'm having issues with my premium subscription",
    thread_id=f"session-1-{test_ticket_id}",
    ticket_id=test_ticket_id
)
print(f"Agent: {result1['response'][:200]}...")

In [ ]:
# Check what was stored in memory
from agentic.tools.memory import get_history, format_history_for_context

print("Checking stored memory...")
print("=" * 50)

history = get_history(test_ticket_id, limit=10)
print(f"Messages stored: {len(history)}")

for msg in history:
    print(f"[{msg['role'].upper()}]: {msg['content'][:100]}...")

In [ ]:
# Simulate a NEW session - but same ticket
# The agent should have access to previous conversation via memory

print("\n[Session 2 - NEW SESSION, SAME TICKET]")
print("=" * 50)

# Different thread_id (new session), same ticket_id (persistent memory)
result2 = ask(
    "So what were we discussing earlier about my subscription?",
    thread_id=f"session-2-{test_ticket_id}",  # New session
    ticket_id=test_ticket_id  # Same ticket - memory should be available
)

print(f"User: So what were we discussing earlier about my subscription?")
print(f"Agent: {result2['response']}")

In [ ]:
# Verify memory was updated with new interaction
print("\nFinal memory state:")
print("=" * 50)

history = get_history(test_ticket_id, limit=10)
print(f"Total messages stored: {len(history)}")

formatted = format_history_for_context(history)
print(formatted)

## 7. Test: Escalation Path

In [ ]:
# Blocked account (auto-escalates)
print("Testing: Blocked account")
print("-" * 40)

result = ask("My account is blocked and I don't know why")

print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
# Explicit human request
print("Testing: Human request")
print("-" * 40)

result = ask("I want to speak to a human please")

print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")

## 8. RAG Semantic Search

In [ ]:
from agentic.tools.rag_tools import search_knowledge_base

# Semantic search test - no exact keywords
print("Query: 'I can't get into my account'")
print("(Should find login articles without 'login' or 'password' keywords)")
print("=" * 50)

result = search_knowledge_base.invoke("I can't get into my account")
print(result[:1000])

## 9. Interactive Chat

In [ ]:
# Interactive chat
# Type your messages and press Enter. Type 'q' to quit.

print("UDA-Hub Support Agent")
print("Hierarchical Architecture: Classifier → Account Lookup → RAG → Resolver/Escalation")
print("Type 'q' to quit")
print("=" * 40)

chat_interface(orchestrator, ticket_id="interactive-session")

## 10. Visualize Workflow

In [ ]:
try:
    from IPython.display import Image, display
    graph_image = orchestrator.get_graph().draw_mermaid_png()
    display(Image(graph_image))
except Exception as e:
    print(f"Could not render graph: {e}")
    print("\nWorkflow structure:")
    print("  START → classify → account_lookup → retrieve_knowledge → [resolve OR escalate] → END")

## Summary

### Architecture: Hierarchical (Supervisor)

### 4 Agents:
1. **Classifier** - Supervisor/Router, categorizes tickets
2. **Account Lookup** - Fetches user context from CultPass DB
3. **Resolver** - Generates responses using RAG
4. **Escalation** - Hands off to human support

### Key Features:
- **Confidence-based escalation**: Low KB match → automatic escalation
- **Long-term memory**: Persists to TicketMessage table across sessions
- **Structured logging**: Full visibility into agent decisions
- **RAG semantic search**: Chroma + OpenAI embeddings

```
User Message
     ↓
┌─────────────────┐
│   CLASSIFIER    │ (Supervisor)
└────────┬────────┘
         ↓
┌─────────────────┐
│ ACCOUNT LOOKUP  │
└────────┬────────┘
         ↓
┌─────────────────┐
│  RAG RETRIEVAL  │ → Confidence Check
└────────┬────────┘
         ↓
   ┌─────┴─────┐
   ↓           ↓
RESOLVE    ESCALATE
```